<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

// 赤チャンネルのRGBカラー範囲（0.8以上）
if (texColor.r >= 0.8 && texColor.g < 0.2 && texColor.b < 0.2)
{
    // 赤チャンネルのRGB範囲に該当するピクセルを処理する
    FragColor = texColor; // 表示するか、他の処理を行う
}
else
{
    discard; // 該当範囲外のピクセルを破棄する
}

In [ ]:

!!ARBfp1.0
TEMP texColor;
TEMP condition;

# 色を読み取る
TEX texColor, fragment.texcoord[0], texture[0], 2D;

# 条件を評価する
SGE condition.x, texColor.r, 0.8;
SLT condition.y, texColor.g, 0.2;
SLT condition.z, texColor.b, 0.2;
MUL condition.x, condition.x, condition.y;
MUL condition.x, condition.x, condition.z;

# 条件を満たさない場合は破棄する
KIL condition.x;

# 条件を満たす場合は出力する
MOV result.color, texColor;

END

In [ ]:

GLint colorLocation = glGetUniformLocation(shaderProgram, "color");
    GLfloat colorValue[3];

  glUseProgram(shaderProgram);

        glUniform3f(colorLocation, 1.0f, 0.0f, 0.0f);
        glGetUniformfv(shaderProgram, colorLocation, colorValue);


        std::cout << "Color: (" << colorValue[0] << ", " << colorValue[1] << ", " << colorValue[2] << ")" << std::endl;

In [ ]:

#include <opencv2/opencv.hpp>
#include <vector>

cv::Mat ConvertToMat(const char* imageData, int width, int height, int channels) {

    return cv::Mat(height, width, CV_8UC(channels), (void*)imageData);
}

int main() {

    const char* imageData = "...";
    int width = 640;
    int height = 480;
    int channels = 3;


    cv::Mat image = ConvertToMat(imageData, width, height, channels);


    std::vector<cv::Mat> slices;
    for (int y = 0; y < image.rows; y += 2) {

        cv::Rect roi(0, y, image.cols, 2);
        slices.push_back(image(roi));
    }


    std::vector<const char*> sliceData;
    for (auto& slice : slices) {
        sliceData.push_back(slice.ptr<const char>());
    }

    const char* temp = sliceData.front();

    //..

    return 0;
}

In [ ]:

#include <opencv2/opencv.hpp>
#include <vector>

int main() {
    // 元の画像を読み込む
    const char* imagePath = "your_image_path.jpg";
    cv::Mat image = cv::imread(imagePath, cv::IMREAD_COLOR);
    if (image.empty()) {
        std::cerr << "画像を読み込めませんでした。" << std::endl;
        return -1;
    }

    std::vector<uchar> buf; // 画像データを保存するバッファ
    for (int y = 0; y < image.rows; y += 2) {
        // ROIを使って画像をスライスする
        cv::Rect roi(0, y, image.cols, 1); // 縦方向に1ピクセル高さでスライス
        cv::Mat slice = image(roi);

        // スライスした画像をバッファに追加
        std::vector<uchar> slice_buf;
        cv::imencode(".jpg", slice, slice_buf); // スライスした画像をJPEG形式でエンコード
        buf.insert(buf.end(), slice_buf.begin(), slice_buf.end());
    }

    // バッファのデータを const char* に変換
    const char* temp = reinterpret_cast<const char*>(buf.data());

    // ここで temp を使用する処理を行う
    // 注意: temp はこのスコープを抜けると無効になるため、必要であれば適切にコピーして使用してください。

    return 0;
}